In [1]:
import string
import pandas as pd
import numpy as np
import re

In [2]:
data_train = pd.read_csv("train.csv")
data_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import pkg_resources
import string
import re
import pickle
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.svm import SVC
from sklearn import svm
from sklearn import pipeline
from sklearn import linear_model

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\madur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\madur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\madur\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# Drop Keyword and Location columns in train and test Datasets

data_train = data_train.drop(['keyword','location'],axis=1)

In [5]:
# removing html code

def remove_html(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    return text

data_train["text"] = data_train["text"].apply(remove_html)
data_train

D:\Anaconda\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


In [6]:
# converting capital letters to simple letters

def text_simple(text):
    text = text.lower() # convert to lowercase
    return text

data_train['text'] = data_train['text'].apply(text_simple)
data_train

,id,text,target
0,1,our deeds are the reason of this #earthquake m...,1
1,4,forest fire near la ronge sask. canada,1
2,5,all residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,just got sent this photo from ruby #alaska as ...,1
...,...,...,...
7608,10869,two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @thetawniest the out of control w...,1
7610,10871,m1.94 [01:04 utc]?5km s of volcano hawaii. htt...,1
7611,10872,police investigating after an e-bike collided ...,1


In [7]:
# removing urls

def remove_urls(text):
    pattern = re.compile(r'https?://(www\.)?(\w+)(\.\w+)(/\w*)?')
    text = re.sub(pattern, "", text)
    return text
data_train["text"] = data_train["text"].apply(remove_urls)
data_train

,id,text,target
0,1,our deeds are the reason of this #earthquake m...,1
1,4,forest fire near la ronge sask. canada,1
2,5,all residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,just got sent this photo from ruby #alaska as ...,1
...,...,...,...
7608,10869,two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @thetawniest the out of control w...,1
7610,10871,m1.94 [01:04 utc]?5km s of volcano hawaii.,1
7611,10872,police investigating after an e-bike collided ...,1


In [8]:
# removing mentions

def remove_mentions(text):
    pattern = re.compile(r"@\w+")
    text = re.sub(pattern, "", text)
    return text
data_train["text"] = data_train["text"].apply(remove_mentions)
data_train

,id,text,target
0,1,our deeds are the reason of this #earthquake m...,1
1,4,forest fire near la ronge sask. canada,1
2,5,all residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,just got sent this photo from ruby #alaska as ...,1
...,...,...,...
7608,10869,two giant cranes holding a bridge collapse int...,1
7609,10870,the out of control wild fires in california ...,1
7610,10871,m1.94 [01:04 utc]?5km s of volcano hawaii.,1
7611,10872,police investigating after an e-bike collided ...,1


In [9]:
# removing punctuations

string.punctuation
def remove_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), "",text)
    return text
data_train["text"] = data_train["text"].apply(remove_punctuations)
data_train

,id,text,target
0,1,our deeds are the reason of this earthquake ma...,1
1,4,forest fire near la ronge sask canada,1
2,5,all residents asked to shelter in place are be...,1
3,6,13000 people receive wildfires evacuation orde...,1
4,7,just got sent this photo from ruby alaska as s...,1
...,...,...,...
7608,10869,two giant cranes holding a bridge collapse int...,1
7609,10870,the out of control wild fires in california ...,1
7610,10871,m194 0104 utc5km s of volcano hawaii,1
7611,10872,police investigating after an ebike collided w...,1


In [10]:
# removing extra spaces

def remove_extra_spaces(text):
    text = re.sub(' +', " ", text).strip()
    return text
data_train["text"] = data_train["text"].apply(remove_extra_spaces)
data_train

,id,text,target
0,1,our deeds are the reason of this earthquake ma...,1
1,4,forest fire near la ronge sask canada,1
2,5,all residents asked to shelter in place are be...,1
3,6,13000 people receive wildfires evacuation orde...,1
4,7,just got sent this photo from ruby alaska as s...,1
...,...,...,...
7608,10869,two giant cranes holding a bridge collapse int...,1
7609,10870,the out of control wild fires in california ev...,1
7610,10871,m194 0104 utc5km s of volcano hawaii,1
7611,10872,police investigating after an ebike collided w...,1


In [11]:
# removing stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return .join([word for word in str(text).split() if word not in stop_words])

data_train["text"] = data_train["text"].apply(remove_stopwords)
data_train

SyntaxError: invalid syntax (299682020.py, line 6)

In [12]:
# lemmatizing

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(words)
    return text
data_train["text"] = data_train["text"].apply(lemmatize_text)
data_train

,id,text,target
0,1,our deed are the reason of this earthquake may...,1
1,4,forest fire near la ronge sask canada,1
2,5,all resident asked to shelter in place are bei...,1
3,6,13000 people receive wildfire evacuation order...,1
4,7,just got sent this photo from ruby alaska a sm...,1
...,...,...,...
7608,10869,two giant crane holding a bridge collapse into...,1
7609,10870,the out of control wild fire in california eve...,1
7610,10871,m194 0104 utc5km s of volcano hawaii,1
7611,10872,police investigating after an ebike collided w...,1


In [13]:
x = data_train['text']
y = data_train['target']

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=4)

In [15]:
from sklearn import feature_extraction
vec = feature_extraction.text.TfidfVectorizer(ngram_range=(1,2),analyzer='char')

In [16]:
model_pipe = pipeline.Pipeline([('vec',vec),('clf',svm.SVC())])

In [17]:
model_pipe.fit(x_train,y_train)

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='char', ngram_range=(1, 2))),
                ('clf', SVC())])

In [18]:
predict_val = model_pipe.predict(x_test)

In [19]:
from sklearn import metrics
metrics.accuracy_score(y_test,predict_val)*100

76.23112278397899

In [20]:
metrics.confusion_matrix(y_test,predict_val)

array([[758, 121],
       [241, 403]], dtype=int64)

In [21]:
model_pipe.predict(['13,000 people receive #wildfires evacuation orders in California '])

array([1], dtype=int64)

In [22]:
new_file = open('train_model.pkl','wb')
pickle.dump(model_pipe,new_file)
new_file.close()